In [1]:
# Import librabry
import os
import re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from num2words import num2words
import spacy
from deep_translator import GoogleTranslator
from underthesea import text_normalize, classify, sentiment
from wordcloud import WordCloud
import emoji

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Read file
cleaned_comment = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "preprocessed", "preprocessed_comment" ,"cleaned_comment_20240408_2129.csv"))

In [2]:
cleaned_comment.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'data_product_id', 'content_comment',
       'lower_comment', 'decoded_comment', 'translated_comment',
       'demojized_comment', 'standardlized_comment', 'cleaned_comment',
       'joined_comment', 'sentiment_label'],
      dtype='object')

In [3]:
cleaned_comment = cleaned_comment.drop("Unnamed: 0.1", axis= 1)
cleaned_comment[["joined_comment", "sentiment_label"]]

,joined_comment,sentiment_label
0,long quality packaging ok,positive
1,like benign support,positive
2,odorless thick consistency apply absorb smooth...,positive
3,innocuous pleasant odorless difficult lid design,negative
4,fast good,positive
...,...,...
5495,feel soft smooth color clear worth money,neutral
5496,raise color dye fail guess dye black,negative
5497,black dye dye smoke black color black tea s su...,positive
5498,purple popular buy try safe color not standard...,neutral


In [4]:
cleaned_comment["sentiment_label"].value_counts()

sentiment_label
positive    4036
neutral      823
negative     641
Name: count, dtype: int64

In [5]:
positive_comments = cleaned_comment[cleaned_comment.sentiment_label == "positive"][:1000]
neutral_negative_comments = cleaned_comment[(cleaned_comment.sentiment_label == "neutral") | (cleaned_comment.sentiment_label == "negative")]

# Kết hợp các DataFrame đã lọc lại
comment = pd.concat([positive_comments, neutral_negative_comments])

In [26]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(comment['cleaned_comment'], comment['sentiment_label'], test_size=0.2, random_state=42)
# Khởi tạo và huấn luyện mô hình TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [27]:
# Khởi tạo và huấn luyện mô hình Logistic Regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_tfidf, y_train)

# Dự đoán trên tập kiểm tra
y_pred = log_reg_model.predict(X_test_tfidf)

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6937119675456389
